In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv', sep = ',')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv', sep = ',')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv', sep = ',')

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.isnull().sum().any()

In [ ]:
test.isnull().sum().any()

In [ ]:
train = train.drop(['id'],axis = 1)
test = test.drop(['id'],axis = 1)

In [ ]:
corr = train.corr()
sns.heatmap(corr)

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
sc = StandardScaler()
le = LabelEncoder()

In [ ]:
X = train.drop(['target'], axis = 1)
y = train['target']
X = sc.fit_transform(X)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 44)

In [ ]:
import lightgbm as lgb
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'learning_rate': 0.01
}
gbm = lgb.LGBMRegressor(**hyper_params)
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(early_stopping=True)
br = BaggingRegressor()
import catboost as cb
import shap
cat = cb.CatBoostRegressor(loss_function='RMSE', random_state = 50,  iterations=3000,learning_rate=0.1,depth=6,l2_leaf_reg=7)
from sklearn.ensemble import HistGradientBoostingRegressor
hgb = HistGradientBoostingRegressor(learning_rate=0.1, max_iter=3000, random_state=45)
dt = DecisionTreeRegressor(max_depth=6, min_samples_split=2,random_state=45)
rf = RandomForestRegressor(n_estimators=700, criterion='squared_error', max_depth=7, random_state = 45)
gbr = GradientBoostingRegressor()

In [ ]:
import xgboost as xg
#regressor=xgboost.XGBRegressor()
xgb = xg.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.12,
             max_delta_step=0, max_depth=6, min_child_weight=1, 
             monotone_constraints='()', n_estimators=100, n_jobs=-1,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=45, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='auto',
             validate_parameters=1, verbosity=0)

In [ ]:
clf = [ hgb, cat]
for model in clf:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2_score = metrics.r2_score(y_test, y_pred)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
    print(model,'\n','RMSE:', RMSE, '\n', "R2Score:", r2_score, '\n')

In [ ]:
sub.head()

In [ ]:
#test = test.drop(['id'], axis = 1)
test = sc.transform(test)

In [ ]:
y_pred = hgb.predict(test)
test_prediction = pd.DataFrame(y_pred, columns=['target'])
ID = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
test_id = ID['id']
ID = pd.DataFrame(test_id, columns=['id'])
result = pd.concat([ID,test_prediction], axis=1)
result.to_csv('submission_hgb.csv',index =False)

In [ ]:
y_pred = cat.predict(test)
test_prediction = pd.DataFrame(y_pred, columns=['target'])
ID = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
test_id = ID['id']
ID = pd.DataFrame(test_id, columns=['id'])
result = pd.concat([ID,test_prediction], axis=1)
result.to_csv('submission_cat.csv',index =False)